## École Polytechnique de Montréal
## Département Génie Informatique et Génie Logiciel

## INF8460 – Traitement automatique de la langue naturelle - TP2

## Objectifs d'apprentissage: 

•	Explorer les modèles d’espace vectoriel (vector space models) comme représentations distribuées de la sémantique des mots 
•	Implémenter la fréquence de co-occurrence et la PPMI
•	Comprendre différentes mesures de distance entre vecteurs de mots 
•	Explorer l’intérêt de la réduction de dimensionnalité 



## Équipe et contributions 
Veuillez indiquer la contribution effective de chaque membre de l'équipe en pourcentage et en indiquant les modules ou questions sur lesquelles chaque membre a travaillé

Cedric Sadeu (1869737): 1/3

Mamoudou Sacko (1924187): 1/3

Oumayma Messoussi (2016797): 1/3



## Support de google Colab



In [1]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!wget https://staff.fnwi.uva.nl/e.bruni/resources/MEN.tar.gz

--2020-10-06 21:27:22--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  17.9MB/s    in 9.3s    

2020-10-06 21:27:31 (8.63 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]

--2020-10-06 21:27:32--  https://staff.fnwi.uva.nl/e.bruni/resources/MEN.tar.gz
Resolving staff.fnwi.uva.nl (staff.fnwi.uva.nl)... 146.50.61.62
Connecting to staff.fnwi.uva.nl (staff.fnwi.uva.nl)|146.50.61.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90373 (88K) [application/x-gzip]
Saving to: ‘MEN.tar.gz’

MEN.tar.gz          100%[===================>]  88.25K  87.9KB/s    in 1.0s    

2020-10-06 21:27:35 (87.9 KB/s) - ‘MEN.tar.gz’ saved [90373/90373]

In [2]:
! tar -xzf aclImdb_v1.tar.gz
! tar -xzf MEN.tar.gz
! mkdir -p vsm

## Librairies externes

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from collections import Counter, defaultdict
from itertools import chain
from bs4 import BeautifulSoup
import csv
import itertools
import numpy as np
import os
import pandas as pd
import glob
import re
import math
import time

from nltk.corpus import stopwords as all_stopwords
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter, defaultdict

from scipy.stats import spearmanr
from scipy.spatial.distance import euclidean, cosine
from IPython.display import display
from sklearn.decomposition import TruncatedSVD

## Valeurs globales

In [5]:
DIRNAME_ACL =  os.path.join(os.getcwd(), "aclImdb/")
DIRNAME_MEN =  os.path.join(os.getcwd(), "MEN/")
DIRNAME_VSM =  os.path.join(os.getcwd(), "vsm")

## 1. Prétraitement (20 points)

**a)**	Le jeu de données est séparé en deux répertoires `train/`et `test`, chacun contenant eux-mêmes deux sous-répertoires `pos/` et `neg/` pour les revues positives et négatives. Un fichier `readme` décrit plus précisément les données. Commencez par lire ces données, en gardant séparées les données d'entraînement et de test. La fonction doit mettre les mots en minuscules,  supprimer les stopwords (vous devez utiliser ceux de NLTK) et afficher le nombre total de phrases d’entrainement,  le nombre total de phrases d’entrainement positives et négatives et le nombre total de phrases de test avec le nombre total de phrases de test positives et négatives ;

In [6]:
train_data = {'id': [], 'text': [], 'rating': [], 'sentiment': []}
test_data = {'id': [], 'text': [], 'rating': [], 'sentiment': []}

for name in ['train/pos', 'train/neg']:
    sentiment = name.split('/')[1]
    for file in glob.glob(DIRNAME_ACL+name+"/*.txt"):
        review_id = int(file.split("/")[-1].split(".")[0].split("_")[0])
        rating = int(file.split("/")[-1].split(".")[0].split("_")[1]) / 10
        f = open(file, encoding="utf8")
        content = f.read()
        
        train_data['id'].append(review_id)
        train_data['text'].append(content)
        train_data['rating'].append(rating)
        train_data['sentiment'].append(sentiment)
                
for name in ['test/pos', 'test/neg']:
    sentiment = name.split('/')[1]
    for file in glob.glob(DIRNAME_ACL+name+"/*.txt"):
        review_id = int(file.split("/")[-1].split(".")[0].split("_")[0])
        rating = int(file.split("/")[-1].split(".")[0].split("_")[1]) / 10
        f = open(file, encoding="utf8")
        content = f.read()
        
        test_data['id'].append(review_id)
        test_data['text'].append(content)
        test_data['rating'].append(rating)
        test_data['sentiment'].append(sentiment)

train_df = pd.DataFrame (train_data, columns = ['id', 'text', 'rating', 'sentiment'])
test_df = pd.DataFrame (test_data, columns = ['id', 'text', 'rating', 'sentiment'])

In [7]:
print(train_df.shape)
train_df.head()

(25000, 4)


,id,text,rating,sentiment
0,4630,This is the second Eytan Fox film I have seen....,0.9,pos
1,10964,"When it first came out, this work by the Meyse...",0.7,pos
2,4696,"even though this movie is quite old, no matter...",1.0,pos
3,4807,I really thought that this movie was superb. N...,1.0,pos
4,5258,"The Ladies Man is laugh out loud funny, with a...",1.0,pos


In [8]:
print(test_df.shape)
test_df.head()

(25000, 4)


,id,text,rating,sentiment
0,5739,Paul Naschy as a ghostly security guard in thi...,0.7,pos
1,10964,"In many ways, the filmic career of independent...",0.7,pos
2,4807,Having seen 'only' about 200 Hong Kong films i...,1.0,pos
3,7259,This movie almost has everything. The action i...,1.0,pos
4,11405,"All you need is great house, a babysitter and ...",0.8,pos


In [10]:
def preprocess(review):
    soup = BeautifulSoup(review, "html.parser")
    text = soup.get_text()
    stop_words = set(all_stopwords.words('english'))
    
    tokens = nltk.word_tokenize(text.lower())
    return [w for w in tokens if w.isalpha() and not w in stop_words] 

In [11]:
start = time.time()
train_df['text'] = train_df['text'].apply(preprocess)
print(time.time() - start)

test_df['text'] = test_df['text'].apply(preprocess)

55.10497570037842


In [12]:
print(train_df.shape)
train_df.head()

(25000, 4)


,id,text,rating,sentiment
0,4630,"[second, eytan, fox, film, seen, fantastic, ac...",0.9,pos
1,10964,"[first, came, work, meysels, brothers, much, c...",0.7,pos
2,4696,"[even, though, movie, quite, old, matter, many...",1.0,pos
3,4807,"[really, thought, movie, superb, history, corr...",1.0,pos
4,5258,"[ladies, man, laugh, loud, funny, great, diver...",1.0,pos


In [13]:
print(test_df.shape)
test_df.head()

(25000, 4)


,id,text,rating,sentiment
0,5739,"[paul, naschy, ghostly, security, guard, scari...",0.7,pos
1,10964,"[many, ways, filmic, career, independent, lege...",0.7,pos
2,4807,"[seen, hong, kong, films, time, say, film, amo...",1.0,pos
3,7259,"[movie, almost, everything, action, cool, funn...",1.0,pos
4,11405,"[need, great, house, babysitter, phone, simon,...",0.8,pos


In [14]:
print("""TRAIN\n\t - le nombre total de phrases d’entrainement: %d
              \n\t - le nombre total de phrases d’entrainement positives: %d
              \n\t - le nombre total de phrases d’entrainement négatives: %d\n""" % 
      (train_df.shape[0], train_df[train_df["sentiment"] == "pos"].shape[0], 
       train_df[train_df["sentiment"] == "neg"].shape[0]))

print("""TEST\n\t - le nombre total de phrases de test: %d
              \n\t - le nombre total de phrases de test positives: %d
              \n\t - le nombre total de phrases de test négatives: %d\n""" % 
      (test_df.shape[0], test_df[test_df["sentiment"] == "pos"].shape[0], 
       test_df[test_df["sentiment"] == "neg"].shape[0]))

TRAIN
	 - le nombre total de phrases d’entrainement: 25000
              
	 - le nombre total de phrases d’entrainement positives: 12500
              
	 - le nombre total de phrases d’entrainement négatives: 12500

TEST
	 - le nombre total de phrases de test: 25000
              
	 - le nombre total de phrases de test positives: 12500
              
	 - le nombre total de phrases de test négatives: 12500



**b)**	Créez la fonction `build_voc()` qui extrait les unigrammes de l’ensemble d’entraînement et conserve ceux qui ont une fréquence d’occurrence d'au moins 5 et imprime le nombre de mots dans le vocabulaire. Sauvegardez-le dans un fichier `vocab.txt` (un mot par ligne) dans le répertoire aclImdb.

In [15]:
def build_voc(corpus, unk_cutoff=5):
    unigrams = defaultdict(int)
    for review in corpus:
        for word in review:
            unigrams[word] += 1

    print("Unigrammes: " + str(len(unigrams)))

    unigrams = {k: v for k, v in sorted(unigrams.items(), key=lambda item: item[1], reverse=True) if v >= 5}

    print("Taille du vocab (apres filtrage freq > 5): " + str(len(unigrams)))

    f = open('vocab.txt', 'w')
    for w in unigrams.keys():
        f.write(w + '\n')

    return unigrams

In [16]:
start = time.time()
train_unigrams = build_voc(train_df['text'])
print(time.time() - start)

Unigrammes: 72029
Taille du vocab (apres filtrage freq > 5): 27760
0.698833703994751


## 2. Matrices de co-occurence (30 points)

Pour les matrices de cette section, vous pourrez utiliser [des array `numpy`](https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html) ou des DataFrame [`pandas`](https://pandas.pydata.org/pandas-docs/stable/). 

Ressources utiles :  le [*quickstart tutorial*](https://numpy.org/devdocs/user/quickstart.html) de numpy et le guide [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html).

**a)** A partir des textes du corpus d’entrainement (neg/pos), vous devez construire une matrice de co-occurrence mot × mot M(w,w) qui contient les 5000 unigrammes les plus fréquents sous forme de **cadre panda**. Le contexte de co-occurrence est une fenêtre de +/-5 mots autour du mot cible. Le poids est la fréquence de co-occurrence simple. Sauvegardez votre matrice dans un fichier tp2_mat5.csv dans le répertoire vsm.

Attention, le mot lui même de doit pas être compté dans sa co-occurence. Exemple : 
Corpus: [ "I go to school every day by bus", "i go to theatre every night by bus"]

Co-occurence("every", fenetre=2) = [ (to, 2), (by, 2), (school, 1), (day, 1), (theatre, 1), (night, 1), (bus, 0), (every, 0), (go, 0). (i,0) ]

In [20]:
train_top_5000 = list(train_unigrams.keys())[:5000] # or read first 5000 lines from vocab.txt
train_5000_df = pd.DataFrame(train_top_5000, columns=['unigram'])
train_5000_df.shape
train_5000_df.head()

,unigram
0,movie
1,film
2,one
3,like
4,good


In [59]:
def matrix_creator(train_reviews, vocab, window_size, scaled):
    M = defaultdict()
    words = []

    for review in train_reviews:
        for token in review:
            if token not in M:
                M[token] = {}
            words.append(token)

        for i in range(len(review)):
            window = [i+k for k in range(-window_size, window_size, 1) if i+k in range(0, len(review)) and i+k != i]

            neighbours_dict = M[review[i]]
            for pos in range(len(window)):
                if review[i] == review[pos]:
                    continue
                if review[pos] not in neighbours_dict:
                    neighbours_dict[review[pos]] = 0

                neighbours_dict[review[pos]] += 1 if scaled == False else 1 / abs(i - pos)
    
    # neighbours_dict = {k: v for k, v in sorted(neighbours_dict.items(), key=lambda item: item[1], reverse=True)}
    # vector = {k: {x: M[k][x] for x in vocab if x in M[k]} for k in vocab}
    
    # tmp = M
    # for w in tmp:
    #     if item not in vocab:
    #         M.pop(item)

    result = [(k, v) for k, v in M.items() if k in vocab]
    df1 = pd.DataFrame(result)
    df2 = pd.json_normalize(df1[1])
    df2 = df2[df1[0]].fillna(0).set_index(df1[0])

    return df2

In [60]:
start = time.time()
Mww = matrix_creator(train_df['text'], train_top_5000, 5, False)
print(time.time() - start)

# Mww = Mww.sort_index(axis=0)
# Mww = Mww.sort_index(axis=1)
print(Mww)

KeyError: ignored

In [126]:
start = time.time()
Mww_scaled = matrix_creator(train_df['text'], train_top_5000, 5, True)
print(time.time() - start)

Mww_scaled = Mww_scaled.sort_index(axis=0)
Mww_scaled = Mww_scaled.sort_index(axis=1)
print(Mww_scaled)

48.63905906677246
           abandoned  abc  abilities  ...     zombie    zombies  zone
abandoned   0.000000  0.0        0.0  ...   0.000000   0.866667  0.00
abc         0.000000  0.0        0.0  ...   0.000000   0.000000  0.00
abilities   0.000000  0.0        0.0  ...   0.000000   0.000000  0.00
ability     0.000000  0.0        0.2  ...   0.750000   0.333333  0.00
able        0.000000  0.0        0.0  ...   0.333333   0.750000  0.00
...              ...  ...        ...  ...        ...        ...   ...
zero        0.000000  0.0        0.0  ...   0.200000   0.000000  0.00
zizek       0.000000  0.0        0.0  ...   0.000000   0.000000  0.00
zombie      0.000000  0.0        0.0  ...   0.000000  15.600000  0.00
zombies     0.666667  0.0        0.0  ...  14.600000   0.000000  0.25
zone        0.000000  0.0        0.0  ...   0.000000   0.450000  0.00

[5000 rows x 5000 columns]


In [123]:
corpus = ["I go to school every day by bus", "I go to theatre every night by bus"]
vocab2 = ['theatre', 'by', 'school', 'day', 'bus']
# print('Vocabulary2:\n',vocab2,'\n')

token_sent_list2 = [word_tokenize(sen) for sen in corpus]
# print('Each sentence in token form2:\n',token_sent_list2,'\n')

start = time.time()
Mww = matrix_creator(token_sent_list2, vocab2, 5, False)
print(time.time() - start)

print(Mww)
df = Mww.sort_index(axis=0)
df = df.sort_index(axis=1)
print(df)

0.0020203590393066406
         theatre   by  school  day  bus
by           1.0  0.0     1.0  1.0  2.0
bus          1.0  2.0     1.0  1.0  0.0
theatre      0.0  1.0     0.0  0.0  1.0
school       0.0  1.0     0.0  1.0  1.0
day          0.0  1.0     1.0  0.0  1.0
         bus   by  day  school  theatre
bus      0.0  2.0  1.0     1.0      1.0
by       2.0  0.0  1.0     1.0      1.0
day      1.0  1.0  0.0     1.0      0.0
school   1.0  1.0  1.0     0.0      0.0
theatre  1.0  1.0  0.0     0.0      0.0


**b)** Calculez maintenant une matrice de cooccurrence mais en ajustant les fréquences basées sur la proximité du mot cible par exemple en les multipliant par 1/𝑑 où d est la distance en jetons (mots) de la cible. Sauvegardez votre matrice (toujours sous forme de cadre panda) dans un fichier tp2_mat5_scaled.csv dans le répertoire vsm.

Voir fonction dessus

**c)**	Vous devez créer une fonction `pmi` qui prend le cadre panda de la matrice $M(w,w)$ et un paramètre boolean flag qui est à True lorsque l'on désire calculer PPMI et à False quand on veut calculer PMI. La fonction transforme la matrice en entrée en une matrice $M’(w,w)$ avec les valeurs PMI ou PPMI selon la valeur du paramètre booléen. La fonction retourne le nouveau cadre panda correspondant. 

Pour une matrice  $X_{m \times n}$:


$$\textbf{colsum}(X, j) = \sum_{i=1}^{m}X_{ij}$$

$$\textbf{sum}(X) = \sum_{i=1}^{m}\sum_{j=1}^{n} X_{ij}$$

$$\textbf{expected}(X, i, j) = 
\frac{
  \textbf{rowsum}(X, i) \cdot \textbf{colsum}(X, j)
}{
  \textbf{sum}(X)
}$$


$$\textbf{pmi}(X, i, j) = \log\left(\frac{X_{ij}}{\textbf{expected}(X, i, j)}\right)$$

$$\textbf{ppmi}(X, i, j) = 
\begin{cases}
\textbf{pmi}(X, i, j) & \textrm{if } \textbf{pmi}(X, i, j) > 0 \\
0 & \textrm{otherwise}
\end{cases}$$


In [137]:
def mat_list_to_df(mat_result, unigramme_list):
    arr = np.array(mat_result)
    t_arr = arr.T
    arr_list = t_arr.tolist()
    dict_result = {}
    for i, word in enumerate(unigramme_list):
        dict_result[word] = arr_list[i]
    df = pd.DataFrame(data = dict_result, index = unigramme_list)
    return df

def probability_matrix(mat_Result):
    total = 0
    for line in mat_Result:
        total += sum(line)
    arr = np.array(mat_Result)
    with np.errstate(divide='ignore'):
        arr_prob = np.divide(arr, total)
    return arr_prob.tolist()

def ppmi_pmi(df_Result, is_ppmi):

    temp = df_Result.to_numpy()
    mat_Result = list(temp.tolist())
    mat_prob = probability_matrix(mat_Result)
    arr = np.array(mat_prob)
    sum_arr_column = arr.sum(axis=0)
    sum_arr_line = arr.sum(axis=1)

    for i, line in enumerate(mat_prob):
        for j, item in enumerate(line):
            if item == 0:
                mat_prob[i][j] = 0.0
            elif is_ppmi:         
                mat_prob[i][j] = max(math.log(item / (sum_arr_column[j]*sum_arr_line[i]), 2), 0)
            else:
                mat_prob[i][j] = math.log(item / (sum_arr_column[j]*sum_arr_line[i]), 2)
    
    return mat_list_to_df(mat_prob, list(df_Result.columns))

**d)** Créer les matrice PMIs et PPMIs en vous basant sur les deux matrices que vous avez déjà créée	Sauvegardez vos matrices dans un fichier tp2_mat5<\_scaled>_{pmi|ppmi}.csv toujours dans le répertoire vsm. 

(votre nom de fichier doit contenir "_scaled" s'il est formé à partir Mww_scaled et "pmi" si le flag est false "ppmi" sinon) 

In [138]:
start = time.time()
result_ppmi_df = ppmi_pmi(Mww, True)
print(time.time() - start)
result_ppmi_df.to_csv('tp2_mat5_ppmi.csv')

start = time.time()
result_pmi_df = ppmi_pmi(Mww, False)
print(time.time() - start)
result_pmi_df.to_csv('tp2_mat5_pmi.csv')

22.18608260154724
19.360100746154785


In [139]:
start = time.time()
result_ppmi_sc_df = ppmi_pmi(Mww_scaled, True)
print(time.time() - start)
result_ppmi_sc_df.to_csv('tp2_mat5_scaled_ppmi.csv')

start = time.time()
result_pmi_sc_df = ppmi_pmi(Mww_scaled, False)
print(time.time() - start)
result_pmi_sc_df.to_csv('tp2_mat5_scaled_pmi.csv')

20.27623438835144
19.46448278427124


In [129]:
result_ppmi_df.head()

,abandoned,abc,abilities,ability,able,abraham,absence,absent,absolute,absolutely,absurd,abuse,abused,abusive,abysmal,academy,accent,accents,accept,acceptable,accepted,access,accident,accidentally,accompanied,accomplished,according,account,accuracy,accurate,accused,achieve,achieved,achievement,acid,across,act,acted,acting,action,...,worthwhile,worthy,would,wound,wounded,wow,wrapped,wreck,wrestling,wretched,write,writer,writers,writes,writing,written,wrong,wrote,wwe,wwii,x,ya,yard,yeah,year,years,yelling,yellow,yes,yesterday,yet,york,young,younger,youth,zero,zizek,zombie,zombies,zone
abandoned,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,3.344372,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.902583,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.881049,1.270665,0.0,0.0,0.357402,0.000000,0.000000,0.0,1.045182,0.0,2.897816,0.000000,0.0,0.000000,3.435152,0.0
abc,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,4.600571,1.842220,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,3.344455,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.146989,0.0,0.000000,0.0,...,0.000000,0.0,0.221416,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,2.026451,0.0,2.056388,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.267360,0.0,0.0,0.939060,0.000000,1.591299,0.0,0.626839,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
abilities,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.076491,0.0,0.000000,0.0,0.0,0.000000,0.0,2.783699,0.000000,0.000000,4.601518,4.472672,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,3.558160,0.0,...,0.000000,0.0,0.192653,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,1.256345,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
ability,0.0,0.0,2.833817,0.000000,1.782457,0.0,0.0,0.000000,2.143693,0.000000,0.0,1.998789,0.0,0.0,0.000000,0.0,1.677512,0.000000,3.272938,0.000000,2.366485,0.000000,0.0,0.000000,0.0,0.000000,1.472539,0.000000,0.0,0.0,2.704258,0.000000,2.417812,2.674474,0.000000,0.000000,0.275073,0.0,2.957864,0.0,...,2.025080,0.0,0.000000,0.0,0.000000,1.094143,0.0,0.0,0.0,0.0,2.154536,0.0,1.769434,0.0,0.000000,0.0,0.293874,1.523555,0.0,0.0,2.335995,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.134421,0.0,0.000000,0.0,0.000000,2.082592,0.0,1.108127,0.559931,0.0
able,0.0,0.0,0.000000,1.997777,0.000000,0.0,0.0,1.769991,0.604859,0.000000,0.0,0.000000,0.0,0.0,1.409815,0.0,0.000000,0.301777,0.734104,0.000000,0.000000,2.516408,0.0,0.461041,0.0,2.099678,0.000000,0.000000,0.0,0.0,1.165424,2.099114,0.000000,0.000000,2.688875,0.000000,1.736239,0.0,0.000000,0.0,...,0.486246,0.0,1.058594,0.0,2.824853,0.000000,0.0,0.0,0.0,0.0,1.423057,0.0,0.645638,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.797161,0.0,0.0,0.0,0.373885,0.441573,0.0,0.0,0.000000,2.866454,0.000000,0.0,0.000000,0.0,1.068723,0.000000,0.0,0.000000,0.021097,0.0


## 3. Test de PPMI (20 points)

Pour le test des matrices de cooccurrences, nous allons comparer deux mesures de distance entre deux vecteurs, la distance euclidienne et la distance cosinus provenant du module [scipy.spatial.distance](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

**Distance Euclidienne**

La distance euclidienne entre deux vecteurs $u$ et $v$ de dimension $n$ est

$$\textbf{euclidean}(u, v) = 
\sqrt{\sum_{i=1}^{n}|u_{i} - v_{i}|^{2}}$$

En deux dimensions, cela correspond à la longueur de la ligne droite entre deux points.

**Distance Cosinus**


La distance cosinus entre deux vecteurs $u$ et $v$ de dimension $n$ s'écrit :

$$\textbf{cosine}(u, v) = 
1 - \frac{\sum_{i=1}^{n} u_{i} \cdot v_{i}}{\|u\|_{2} \cdot \|v\|_{2}}$$

Le terme de droite dans la soustraction mesure l'angle entre $u$ et $v$; on l'appelle la *similarité cosinus* entre $u$ et $v$.

\\

**a)**	Implémentez la fonction voisins(mot, pd, distance) qui prend un mot en entrée et une métrique de distance et qui retourne les n mots les plus similaires selon la mesure. Pour un mot w, elle ordonne tous les mots du vocabulaire en fonction de leur distance de w en utilisant la métrique de distance distance (par défaut: cosine)sur le vsm pd. Les mesures de distance à tester sont : la distance Euclidienne et la distance cosinus implantées ci-dessus.

In [134]:
def voisins(word, df, n, distfunc=cosine):
    assert distfunc.__name__ == 'cosine' or distfunc.__name__ == 'euclidean', "distance metric not supported"
    assert word in df.columns, "word not in df"
    order = True if distfunc.__name__ == 'euclidean' else False

    closest = {}
    for w in df.columns:
        if w == word:
            continue
        distance = distfunc(df[word], df[w])
        closest[w] = distance
    closest = {k: v for k, v in sorted(closest.items(), key=lambda item: item[1], reverse=order)}

    return list(closest.keys())[:n], list(closest.values())[:n]

In [ ]:
Mtest = pd.DataFrame([[2.0, 10.0, 14.0], [6.0, 14.0, 9.0]], columns=['excellent', 'bon', 'decevant'])
print(Mtest.head())
print(voisins('bon', Mtest, 5, euclidean))
print(voisins('bon', Mtest, 5, cosine))

   excellent   bon  decevant
0        2.0  10.0      14.0
1        6.0  14.0       9.0
(['excellent', 'decevant'], [11.313708498984761, 6.4031242374328485])
(['excellent', 'decevant'], [0.044220991278050015, 0.0710423905158224])


**b)** En utilisant le cadre panda associé aux matrices Mww et Mww scaled, trouvez les 5 mots les plus similaires au mot « beautiful » et affichez-les, pour chacune des deux distances.

In [135]:
print(voisins('beautiful', Mww, 5, euclidean))
print(voisins('beautiful', Mww_scaled, 5, cosine))

(['movie', 'film', 'one', 'like', 'good'], [12819.290932028964, 10146.900314874489, 7756.188883723758, 6364.71609736051, 5405.231724912448])
(['amazing', 'also', 'however', 'wonderful', 'although'], [0.1658613382003148, 0.1698264792866262, 0.17868317527019228, 0.18665683058421279, 0.18716703583285477])


**c)** En utilisant les cadres panda associés aux matrices PMIs, trouvez les 5 mots les plus similaires au mot « beautiful » et affichez-les, pour chacune des deux distances.

In [142]:
print(voisins('beautiful', result_pmi_df, 5, euclidean))
print(voisins('beautiful', result_pmi_df, 5, cosine))

(['bad', 'orders', 'officer', 'discovers', 'security'], [101.4788676265592, 100.32576475413418, 100.15517855117686, 100.0009727143084, 99.7279357750562])
(['wonderful', 'cinematography', 'perfect', 'beauty', 'gorgeous'], [0.6717705829720779, 0.6951024923704886, 0.7023460503696113, 0.7024872148757113, 0.7031228628702768])


**d)** En utilisant les cadres panda associés aux matrices PPMIs, trouvez les 5 mots les plus similaires au mot
« beautiful » et affichez-les, pour chacune des deux distances. 

In [143]:
print(voisins('beautiful', result_ppmi_df, 5, euclidean))
print(voisins('beautiful', result_ppmi_df, 5, cosine))

(['orders', 'chief', 'rival', 'discovers', 'colonel'], [89.755826104396, 89.15646658486772, 89.1203669419272, 89.10852246934506, 89.09471338686936])
(['gorgeous', 'wonderful', 'beauty', 'cinematography', 'woman'], [0.593391015475159, 0.6072324707509398, 0.614842595490545, 0.6163763362061689, 0.6175146843785463])


**e)** Que constatez-vous entre la différence de performance de la distance euclidienne et la distance cosinus ? Que constatez-vous entre les différents types de matrices de cooccurrence ?

## 4.	Réduction de dimensionnalité (20 points)

**a)** Ecrivez une fonction lsa qui prend en entrée un cadre panda pd (qui contient votre matrice / vsm) et un paramètre K (qui indique le nombre de dimensions finales), et qui applique LSA avec ce paramètre k sur la matrice et retourne le vsm réduit sous forme de cadre panda.

In [ ]:
mots = ['red', 'blue', 'green', 'yellow', 'orange', 'black']
data = np.array([ [1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0], [1, 0, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1]])
vsm = pd.DataFrame(data,  columns=mots)
PMIs = pd.DataFrame(data,  columns=mots)
PPMIs = pd.DataFrame(data, columns=mots)

In [144]:
def lsa(df, k):
    U, s, V = np.linalg.svd(df, full_matrices=False)
    S = np.diag(s[:k])
    Ck = np.dot(U[:, :k], S)
    print(Ck)
    return pd.DataFrame(Ck)

# display(lsa(vsm, 3))

**b)** Exécutez lsa sur les cadres panda associés à vos matrices Mww et Mww_scaled avec une dimension k=100

In [146]:
Mww_lsa = lsa(Mww, 100)
Mww_scaled_lsa = lsa(Mww_scaled, 100)

[[ -41.01546387   -3.92034505    5.20777349 ...    4.28541938
    -4.59657506    4.11725548]
 [ -29.35567958    8.73518688   -0.52875177 ...   -0.63410995
    -1.45120792   -3.43201123]
 [ -24.77290098   -0.73145729    4.32052535 ...   -0.23228785
     0.70248138   -3.19548068]
 ...
 [-220.15132425  -38.44048945  -10.73317571 ...   -0.43295407
    10.16858876   -4.71748159]
 [-159.13755874  -17.14520653   12.25246359 ...   -0.35241375
    11.17296879   -5.20608722]
 [ -31.19791467    4.16886182    1.28194283 ...    2.07758977
     0.65691439   -0.33459172]]
[[-1.77360015e+01 -1.84551872e+00 -1.09727495e+00 ... -1.27228970e+00
   2.58505671e-02  1.57513359e+00]
 [-1.37109928e+01  3.93208644e+00 -4.56979046e-01 ...  4.35868439e-01
   9.98178934e-01  1.31036552e+00]
 [-1.19404736e+01 -8.56073877e-01 -4.27252654e+00 ...  1.01750377e+00
   1.51038100e-01 -3.02578445e-01]
 ...
 [-1.15844884e+02 -3.41489229e+01  1.32056160e+01 ... -2.75631653e+00
  -3.95319370e-01  5.14576106e-01]
 [-7.414528

**c)** En utilisant les matrices de co-occurrence (de base et scalés) réduites avec LSA, trouvez les 5 mots les plus similaires au mot « beautiful » selon la distance cosinus et affichez-les.

In [147]:
print(voisins('beautiful', Mww_lsa, 5, euclidean))
print(voisins('beautiful', Mww_scaled_lsa, 5, cosine))

AssertionError: ignored

d) En utilisant les matrices PMIs et PPMIs réduites avec lsa, trouvez les 5 mots les plus similaires au mot « beautiful » selon la distance cosinus et affichez-les

In [ ]:
ppmi_lsa_df = ppmi_pmi(Mww_lsa, True)
ppmi_lsa_scaled_df = ppmi_pmi(Mww_scaled_lsa, True)
pmi_lsa_df = ppmi_pmi(Mww_lsa, False)
pmi_lsa_scaled_df = ppmi_pmi(Mww_scaled_lsa, False)

print(voisins('beautiful', ppmi_lsa_df, cosine)[:5])
print(voisins('beautiful', ppmi_lsa_scaled_df, cosine)[:5])
print(voisins('beautiful', pmi_lsa_df, cosine)[:5])
print(voisins('beautiful', pmi_lsa_scaled_df, cosine)[:5])

**e)** En utilisant sklearn.decomposition.TruncatedSVD, créez les matrices réduites à partir des mêmes matrices que celles de la question précédentes (la matrice pmi et la matrice pmi_scaled) Puis tester ces nouvelles matrices LSA pour trouver les 5 mots les plus similaires au mot « beautiful » 

Ici aussi, nous voulons aussi obtenir des matrices de dimension k=100

In [ ]:
PMIs = pd.DataFrame(data, columns=mots)
PPMIs = pd.DataFrame(data, columns=mots)

svd_model = TruncatedSVD(n_components=5, algorithm='randomized', n_iter=100, random_state=122)

PMIs = svd_model.fit_transform(PMIs)
PPMIs = svd_model.fit_transform(PPMIs)

display(PMIs)
display(PPMIs)

f) Commentez vos résultats

## 5. Évaluation (10 points)

Il est temps d’évaluer l’intérêt de nos modèles de vecteurs. Nous allons pour cela utiliser un ensemble de données de similarité de mots (relatedness) The MEN Test Collection, qui se trouve dans le répertoire test. L’ensemble de données contient une paire de mots avec un score de similarité attribué par des humains. En d’autres termes, une ligne (un exemple) de l’ensemble de données est de la forme : \<mot_1> \<mot_2> \<score>.

Pour aligner les distances obtenues avec vos métriques, ce score est converti en nombre réel négatif par la fonction read_test_dataset que vous avez dans le squelette du TP.

La métrique d’évaluation est le coefficient de corrélation de Spearman 𝜌 entre les scores humains et vos distances (voir https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient). 

Nous allons maintenant évaluer les différents vsm obtenus sur l'ensemble de données: MEN_dataset.


#### Fonctions pour lire le jeu de données MEN

In [ ]:
def read_test_dataset(
        src_filename,
        delimiter=','):
    with open(src_filename) as f:
        reader = csv.reader(f, delimiter=delimiter)
        for row in reader:
            w1 = row[0].strip().lower()
            w2 = row[1].strip().lower()
            score = row[2]
            score = -float(score)
            yield (w1, w2, score)

In [ ]:
# Retourne un itérable sur le jeu de données MEN
def men_dataset():
    src_filename = os.path.join(
        DIRNAME_MEN, 'MEN_dataset_natural_form_full')
    return read_test_dataset(
        src_filename, delimiter=' ')

In [ ]:
def evaluate(ds, df, distfunc=cosine):
    """
    ds : iterator
       retourne des tuples (word1, word2, score).

    df : pd.DataFrame
        le modèle vsm à évaluer

    distfunc : la mesure de distance entre vecteurs
  
    Retour: le coefficient de correlation de Spearman entre les scores de l'ensemble de données de test 
    et celui du modele vsm qui se présente sous la forme d'un cadre Panda pd avec les colonnes
    ['word1', 'word2', 'score', 'distance'].
    """
    data = []
    for w1, w2, score in ds():
        d = {'word1': w1, 'word2': w2,'score': score}
        if w1 not in df.index or w2 not in df.index:
            continue
        else:
            w1 = df.loc[w1]
            w2 = df.loc[w2] 
        d['distance'] = distfunc(w1, w2)
        data.append(d)

    data = pd.DataFrame(data)
    rho, pvalue = spearmanr(data['score'].values, b=data['distance'].values)
    return rho, data

**a)**	Testez chacun de vos modèles vsm (Matrice de base, matrice scalée, les PMIs et PPMIs et toutes les matrices LSA (de base, scalée, pmi, ppmi) en appelant la fonction evaluate avec les deux mesure de distance (euclidienne et cosinus) et affichez vos résultats dans une seule table.

**b)**	Commentez vos résultats d'évaluation